# Foursquare dataset next-POI Recommendation System

First off we import all the necessary libraries:

In [24]:
%pip install lightning geohash2

  Preparing metadata (setup.py) ... done
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15543 sha256=bb500658b5569c6699056cd94a1dcb4764a4beeb9575463869afb672e3534fe0
  Stored in directory: /root/.cache/pip/wheels/c0/21/8d/fe65503f4f439aef35193e5ec10a14adc945e20ff87eb35895
Successfully built geohash2
Note: you may need to restart the kernel to use updated packages.


In [2]:
import polars as rs
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as pl
import lightning.pytorch as torchpl
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder
from dataclasses import dataclass
import wandb

In [3]:
import os

# define WANDB_NOTEBOOK_NAME
os.environ["WANDB_NOTEBOOK_NAME"] = "train.ipynb"

In [4]:
import gc

gc.collect()
# clean CUDA memory
torch.cuda.empty_cache()

# sometimes jupyter notebook does not release memory, we leave this here so a run-all
# can *sometimes* fix leaks

Next, we load the data, we utilize `polars` since it is much more efficient than `pandas` and can handle large datasets with ease.

In [5]:
columns = ["user", "poi", "date", "TZ"]
data = rs.read_csv(
    "/kaggle/input/dataset-tist2015/dataset_TIST2015_Checkins.txt",
    has_header=False,
    low_memory=True,
    separator="\t",
)
data.columns = columns

In [6]:
data

user,poi,date,TZ
i64,str,str,i64
50756,"""4f5e3a72e4b053…","""Tue Apr 03 18:…",240
190571,"""4b4b87b5f964a5…","""Tue Apr 03 18:…",180
221021,"""4a85b1b3f964a5…","""Tue Apr 03 18:…",-240
66981,"""4b4606f2f964a5…","""Tue Apr 03 18:…",-300
21010,"""4c2b4e8a9a559c…","""Tue Apr 03 18:…",240
…,…,…,…
16349,"""4c957755c8a1bf…","""Mon Sep 16 23:…",-240
256757,"""4c8bbb6d9ef022…","""Mon Sep 16 23:…",-180
66425,"""513e82a5e4b0ed…","""Mon Sep 16 23:…",-180


Differently from what suggested by the professor, we utilize the full TIST2015 dataset, which has a far greater scale compared to the reduced NY one. However, by following the pruning steps detailed in the paper (http://dx.doi.org/10.1145/3477495.3531989, section 5.1), we obtain sequences that are much smaller in size, resulting in a dataset that is usable on Google Colab's free tier (as required by the assignment).

In [7]:
data_users = (
    data.lazy()
    .group_by("user")
    .agg(
        [
            rs.col("poi").n_unique().alias("n_pois"),
            rs.col("poi").count().alias("n_checkins"),
            # turn the rest into a list
            rs.col("poi").alias("pois"),
            rs.col("date").alias("dates"),
            rs.col("TZ").alias("TZs"),
        ]
    )
).collect()

In [8]:
data_users.describe()

statistic,user,n_pois,n_checkins,pois,dates,TZs
str,f64,f64,f64,f64,f64,f64
"""count""",266909.0,266909.0,266909.0,266909.0,266909.0,266909.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",133455.0,56.477459,124.62537,null,null,null
"""std""",77050.135837,45.968603,140.692138,null,null,null
"""min""",1.0,1.0,1.0,null,null,null
"""25%""",66728.0,30.0,61.0,null,null,null
"""50%""",133455.0,49.0,93.0,null,null,null
"""75%""",200182.0,71.0,148.0,null,null,null
"""max""",266909.0,1246.0,5430.0,null,null,null


## Data Preprocessing

In [9]:
data_culled = data_users.filter(
    (rs.col("n_checkins") > 20) & (rs.col("n_checkins") < 50)
).drop_nulls()

Since the original dataset is huge, we delete it and call the python garbage collector to free up memory. We then proceed with the second pruning step (frequency-based pruning) as detailed in the paper.

In [10]:
del data
del data_users

import gc

gc.collect()

0

In [11]:
# extract unique elements from each lists in data_culled["pois"]
out = data_culled.with_columns(
    [
        rs.col("pois").list.unique(),
        rs.col("pois").list.unique().list.len().alias("n_unique_pois"),
    ]
)

In [12]:
out

user,n_pois,n_checkins,pois,dates,TZs,n_unique_pois
i64,u32,u32,list[str],list[str],list[i64],u32
132850,31,37,"[""4b5a9b91f964a52087cd28e3"", ""4edd7cb493ad464d2c35afa3"", … ""4e54e79efa761d3de841cdbb""]","[""Sat Aug 11 03:55:59 +0000 2012"", ""Sat Aug 11 04:37:03 +0000 2012"", … ""Sun Sep 08 00:00:32 +0000 2013""]","[540, 540, … 540]",31
213007,11,22,"[""50439ff6e4b072dc96e9a554"", ""4d4daf82b887a1cd6dc7bea0"", … ""4c2639c2f1272d7fd02286c5""]","[""Sun Sep 09 16:51:33 +0000 2012"", ""Thu Sep 13 13:41:31 +0000 2012"", … ""Thu Aug 08 21:34:50 +0000 2013""]","[180, 180, … 240]",11
134696,22,23,"[""4b42f1d3f964a52024db25e3"", ""4c8cd18ded3ab60c40d56a21"", … ""4d7a8455e8b7a1cde1a2991f""]","[""Thu Jul 05 22:34:13 +0000 2012"", ""Fri Jul 13 01:09:10 +0000 2012"", … ""Wed Aug 14 07:17:39 +0000 2013""]","[-420, -300, … -420]",22
142913,35,45,"[""4cccdbbb7c2ff04d4d179c7e"", ""4e48e75745dd2955e5b74023"", … ""4b6b937ef964a520d4102ce3""]","[""Wed Oct 24 00:39:20 +0000 2012"", ""Mon Oct 29 05:38:02 +0000 2012"", … ""Sun Sep 01 01:02:25 +0000 2013""]","[540, 540, … 540]",35
80691,21,21,"[""4c0e115bc700c9b6ce42a3dd"", ""4b8cf019f964a52079e132e3"", … ""4cb32e59db32f04d265dd54d""]","[""Sat May 19 09:08:52 +0000 2012"", ""Sat May 19 09:41:10 +0000 2012"", … ""Fri Aug 02 18:27:24 +0000 2013""]","[180, 180, … 180]",21
…,…,…,…,…,…,…
189251,28,48,"[""4e39b273b61c05fb6781423f"", ""4a735cabf964a52043dc1fe3"", … ""4e16047718a8addae8cf9047""]","[""Sat Apr 28 04:42:33 +0000 2012"", ""Sat Apr 28 23:53:28 +0000 2012"", … ""Wed Aug 21 18:55:11 +0000 2013""]","[-420, -420, … 180]",28
176237,35,35,"[""4b0587d1f964a520e2a222e3"", ""4fce3376e4b0409de3d1eadc"", … ""4f88874e121d29fd93b03c92""]","[""Wed Mar 06 14:17:09 +0000 2013"", ""Wed Mar 06 14:17:26 +0000 2013"", … ""Thu Mar 07 16:57:52 +0000 2013""]","[-180, -180, … -180]",35
141132,16,36,"[""4fae6095e4b0d39d5300b8d4"", ""5027843ae4b05db3aefaada4"", … ""4bc8a4b815a7ef3b61897bda""]","[""Sun Aug 12 16:15:20 +0000 2012"", ""Fri Aug 17 14:18:59 +0000 2012"", … ""Tue Apr 30 11:03:23 +0000 2013""]","[180, 180, … 180]",16


In [13]:
l = out["pois"][0].to_list()
len(set(l))  # print number of unique POIs in first sequence

31

In [14]:
l2 = data_culled["pois"][0].to_list()
len(l2)  # print sequence length of first user

37

In [15]:
len(set(l2))  # confirm that the two match

31

In [16]:
# run a Polars query to obtain all the frequent POIs, the ones expected to survive the filtering
unique_pois = out["pois"]
frequent_pois = unique_pois.list.explode().value_counts().filter(rs.col("count") >= 10)

In [17]:
frequent_pois

pois,count
str,u32
"""4ba12c74f964a5…",15
"""4c0f537ed64c0f…",20
"""4bbb5abc935e95…",15
"""4af53ef1f964a5…",26
"""506964fae4b03c…",62
…,…
"""4acbc3fbf964a5…",11
"""4cade9aa8b57a1…",15
"""4e7d9d98f5b922…",12


In [18]:
frequent_pois = frequent_pois["pois"]
frequent_pois = set(frequent_pois.to_list())

In [19]:
data_culled

user,n_pois,n_checkins,pois,dates,TZs
i64,u32,u32,list[str],list[str],list[i64]
132850,31,37,"[""4b396b34f964a5204f5c25e3"", ""4b4162b1f964a52016c625e3"", … ""4b5ab7bff964a520eed128e3""]","[""Sat Aug 11 03:55:59 +0000 2012"", ""Sat Aug 11 04:37:03 +0000 2012"", … ""Sun Sep 08 00:00:32 +0000 2013""]","[540, 540, … 540]"
213007,11,22,"[""4b69377ff964a520f59b2be3"", ""4c2639c2f1272d7fd02286c5"", … ""4d4daf82b887a1cd6dc7bea0""]","[""Sun Sep 09 16:51:33 +0000 2012"", ""Thu Sep 13 13:41:31 +0000 2012"", … ""Thu Aug 08 21:34:50 +0000 2013""]","[180, 180, … 240]"
134696,22,23,"[""4c2cdcd2ae6820a102041a43"", ""4b42f1d3f964a52024db25e3"", … ""50065de8e4b00dc76c134f45""]","[""Thu Jul 05 22:34:13 +0000 2012"", ""Fri Jul 13 01:09:10 +0000 2012"", … ""Wed Aug 14 07:17:39 +0000 2013""]","[-420, -300, … -420]"
142913,35,45,"[""4b95a8b4f964a52036ae34e3"", ""4b49ac6bf964a5205c7226e3"", … ""4cbbd0574352a1cdf9039bf5""]","[""Wed Oct 24 00:39:20 +0000 2012"", ""Mon Oct 29 05:38:02 +0000 2012"", … ""Sun Sep 01 01:02:25 +0000 2013""]","[540, 540, … 540]"
80691,21,21,"[""4d122dae7a8ba1433410c769"", ""4cbb569ebac937046d7af17c"", … ""4d41bcd0915a37046104237f""]","[""Sat May 19 09:08:52 +0000 2012"", ""Sat May 19 09:41:10 +0000 2012"", … ""Fri Aug 02 18:27:24 +0000 2013""]","[180, 180, … 180]"
…,…,…,…,…,…
189251,28,48,"[""4bfb3261d0382d7f8d6ec90a"", ""4a735cabf964a52043dc1fe3"", … ""5203e880498e9649d09ff652""]","[""Sat Apr 28 04:42:33 +0000 2012"", ""Sat Apr 28 23:53:28 +0000 2012"", … ""Wed Aug 21 18:55:11 +0000 2013""]","[-420, -420, … 180]"
176237,35,35,"[""4b605cadf964a520c4e129e3"", ""50b93759e4b0073faaf993de"", … ""4e21fcb77d8b71715bb13cde""]","[""Wed Mar 06 14:17:09 +0000 2013"", ""Wed Mar 06 14:17:26 +0000 2013"", … ""Thu Mar 07 16:57:52 +0000 2013""]","[-180, -180, … -180]"
141132,16,36,"[""5027843ae4b05db3aefaada4"", ""4fae6095e4b0d39d5300b8d4"", … ""4e801da54690084c2eaf2291""]","[""Sun Aug 12 16:15:20 +0000 2012"", ""Fri Aug 17 14:18:59 +0000 2012"", … ""Tue Apr 30 11:03:23 +0000 2013""]","[180, 180, … 180]"


In [20]:
data_culled = data_culled.with_columns(
    [
        rs.col("pois")
        .list.eval(
            rs.element().is_in(frequent_pois),
        )
        .alias("is_frequent")
    ]
)  # prep mask

In [21]:
final_data = (
    data_culled.lazy()
    .with_row_index()
    .explode(
        [
            "pois",
            "dates",
            "TZs",
            "is_frequent",
        ]
    )
    .group_by("user")
    .agg(
        [
            rs.col("pois").filter(rs.col("is_frequent")).alias("pois"),
            rs.col("dates").filter(rs.col("is_frequent")).alias("dates"),
            rs.col("TZs").filter(rs.col("is_frequent")).alias("TZs"),
            rs.col("pois").filter(rs.col("is_frequent")).n_unique().alias("n_pois"),
            rs.col("pois").filter(rs.col("is_frequent")).count().alias("n_checkins"),
        ]
    )
    .filter(rs.col("n_checkins") > 0)
    .filter(rs.col("n_pois") > 0)
    .collect()
)  # filter out infrequent pois and users with no pois

In [22]:
final_data.describe()

statistic,user,pois,dates,TZs,n_pois,n_checkins
str,f64,f64,f64,f64,f64,f64
"""count""",19862.0,19862.0,19862.0,19862.0,19862.0,19862.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",156852.822274,null,null,null,6.123452,8.831437
"""std""",76314.892884,null,null,null,4.609024,6.877662
"""min""",49.0,null,null,null,1.0,1.0
"""25%""",95613.0,null,null,null,3.0,4.0
"""50%""",167846.0,null,null,null,5.0,7.0
"""75%""",224576.0,null,null,null,8.0,12.0
"""max""",266909.0,null,null,null,32.0,46.0


At this stage, culling is done, we can appreciate that `polars`'s SQL/functional-style API is different from Pandas, but it is very powerful and efficient.

The next step is geohashing the POIs, that is, we want to convert the latitude-longitude positions of the POIs into a grid-based geohash representation, which will form the basis for our network's embeddings.

In [33]:
import geohash2 as gh

pois = rs.read_csv(
    "/kaggle/input/dataset-tist2015/dataset_TIST2015_POIs.txt",
    has_header=False,
    low_memory=True,
    separator="\t",
)
pois.columns = ["poi", "lat", "long", "category", "country"]
pois = pois.drop("category").drop("country")

In [34]:
pois = (
    pois.lazy()
    .filter(rs.col("poi").is_in(frequent_pois))
    .select(
        [
            rs.col("poi"),
            rs.struct(
                [
                    rs.col("lat").cast(rs.Float32),
                    rs.col("long").cast(rs.Float32),
                ]
            )
            .alias("location")
            .map_elements(
                lambda s: gh.encode(s["lat"], s["long"], precision=6),
                return_dtype=rs.String,
            )
            .alias("geohash"),
        ]
    )
    .collect()
)
poi_geo_dict = dict(zip(pois["poi"], pois["geohash"]))

In [35]:
# for each row in final_data, add the geohash of the pois by hitting the poi_geo_dict

final_data = final_data.with_columns(
    [
        rs.col("pois")
        .map_elements(
            lambda s: [poi_geo_dict[s] for s in s],
            return_dtype=rs.List(rs.String),
        )
        .alias("geohashes")
    ]
)

In [36]:
final_data["dates"][79].to_list()  # check out a temporal sequence

['Mon Mar 04 16:46:12 +0000 2013',
 'Fri Mar 08 15:44:35 +0000 2013',
 'Tue Mar 19 16:53:01 +0000 2013',
 'Fri Mar 22 10:10:18 +0000 2013',
 'Fri Apr 19 15:41:56 +0000 2013',
 'Mon Apr 22 13:11:07 +0000 2013',
 'Mon May 06 11:40:38 +0000 2013',
 'Sat Jun 01 12:55:04 +0000 2013',
 'Wed Jul 10 10:04:08 +0000 2013',
 'Sun Jul 14 09:16:57 +0000 2013',
 'Sun Aug 11 19:00:35 +0000 2013',
 'Tue Aug 13 12:46:54 +0000 2013',
 'Tue Aug 13 13:13:21 +0000 2013',
 'Wed Aug 14 09:55:38 +0000 2013']

In [37]:
final_data["TZs"][79].to_list()  # ... and the corresponding timezones

[120, 120, 120, 120, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180]

The work *might* seem over, however, we still have timezones to account for, we want to normalize everything according to GMT, so we convert the timestamps accordingly.

In [38]:
import datetime


def UTC_to_local(utc, tz):

    date = datetime.datetime.strptime(utc, "%a %b %d %H:%M:%S %z %Y")
    date = date.replace(tzinfo=datetime.timezone.utc)

    # shift by tz offset
    date = date.astimezone(datetime.timezone(datetime.timedelta(minutes=tz)))

    date_s = datetime.datetime.strftime(date, "%Y-%m-%d %H:%M:%S")
    return date_s


def to_UNIX_time(date):
    return datetime.datetime.strptime(
        date, "%Y-%m-%d %H:%M:%S"
    ).timestamp()  # we use UNIX time as a key for sorting the POIs in our polars query

In [39]:
UTC_to_local("Mon May 21 15:53:01 +0000 2012", -420)  # example of usage

'2012-05-21 08:53:01'

In [40]:
final_data = final_data.with_columns(
    [
        rs.struct([rs.col("dates"), rs.col("TZs")])
        .alias("times")
        .map_elements(
            lambda struct: [
                UTC_to_local(date, tz)
                for date, tz in zip(struct["dates"], struct["TZs"])
            ],
            return_dtype=rs.List(rs.String),
        )
    ]
)  # This performs timezone conversion

In [41]:
final_sorted = final_data.select(  # sort the times
    [
        rs.col("user"),
        rs.struct(
            [
                rs.col("pois"),
                rs.col("times"),
            ]
        ).map_elements(
            lambda struct: [
                poi
                for poi, _ in sorted(
                    zip(  # here we sort the POIs struct by UNIX timestamps of the GMT times
                        struct["pois"], [to_UNIX_time(date) for date in struct["times"]]
                    ),
                    key=lambda s: s[1],
                )
            ],
            return_dtype=rs.List(rs.String),
        ),
        rs.struct(
            [
                rs.col("geohashes"),
                rs.col("times"),
            ]
        ).map_elements(
            lambda struct: [
                geo
                for geo, _ in sorted(
                    zip(
                        struct["geohashes"],  # same thing goes on for geohashes
                        [to_UNIX_time(date) for date in struct["times"]],
                    ),
                    key=lambda s: s[1],
                )
            ],
            return_dtype=rs.List(rs.String),
        ),
        rs.col("times")
        .map_elements(
            lambda dates: sorted(dates, key=to_UNIX_time),
            return_dtype=rs.List(rs.String),
        )
        .alias("times_sorted"),
        rs.col("n_checkins"),
    ]
)

# P.S, admittedly, it would have been more efficient to encode the geohashes *after* sorting the POIs,
# so that we could save on the sorting of the geohashes. Tough luck, you can't win 'em all.

In [42]:
final_sorted

user,pois,geohashes,times_sorted,n_checkins
i64,list[str],list[str],list[str],u32
170995,"[""4fc7c357e4b026df79ac7b72"", ""4bc4f2f80191c9b6c7e0eab1"", … ""4bc4f2f80191c9b6c7e0eab1""]","[""ucftph"", ""ucfv2c"", … ""ucfv2c""]","[""2012-06-01 12:52:49"", ""2012-06-07 14:47:23"", … ""2013-04-15 14:11:22""]",8
216178,"[""4bc09456461576b090507a32"", ""4c45f30dcd5ca5930962ff86"", … ""4c03550b187ec9283a97b57b""]","[""swtccd"", ""swtcfd"", … ""sy89km""]","[""2012-04-14 15:19:44"", ""2012-04-14 20:04:43"", … ""2012-07-06 14:10:03""]",12
253183,"[""4af53ef1f964a520a8f821e3""]","[""u09tvn""]","[""2012-05-07 16:37:14""]",1
178066,"[""4c6bdfaa69b4ef3ba18d474e"", ""4db059288154905aadac64ba"", … ""4b490ccef964a520aa6326e3""]","[""7nyywt"", ""7nyyyy"", … ""7nyyyy""]","[""2012-10-17 07:35:16"", ""2012-12-18 12:48:52"", … ""2013-08-26 13:43:01""]",6
199029,"[""4a5920b8f964a520bfb81fe3"", ""4a5920b8f964a520bfb81fe3"", … ""45840abff964a520913f1fe3""]","[""9vg4vz"", ""9vg4vz"", … ""dp3wq6""]","[""2012-04-07 16:33:46"", ""2012-04-19 16:11:55"", … ""2013-06-22 14:49:30""]",21
…,…,…,…,…
246013,"[""4b57d346f964a520e24228e3"", ""4b57d346f964a520e24228e3"", … ""4a5d7fa0f964a520a9bd1fe3""]","[""9v4p2j"", ""9v4p2j"", … ""9x225b""]","[""2012-11-05 19:25:48"", ""2013-01-16 19:33:32"", … ""2013-08-04 16:33:55""]",5
221358,"[""4b617741f964a52082142ae3"", ""4b6ea53df964a52065c52ce3"", … ""4bc2b4e1461576b0e5c77d32""]","[""qquj02"", ""qquj03"", … ""qqgvnn""]","[""2013-04-12 15:35:15"", ""2013-04-25 16:08:46"", … ""2013-06-06 15:39:17""]",6
201577,"[""4b469e61f964a520aa2526e3"", ""4b9bae23f964a5208d1936e3"", … ""4b469e61f964a520aa2526e3""]","[""9zmk3r"", ""9mudgk"", … ""9zmk3r""]","[""2012-05-18 19:29:08"", ""2012-08-14 16:11:37"", … ""2012-11-23 02:50:52""]",6


we now need to obtain a dataframe containing: each POI, it's geohash, and a set of all the check-ins it appears in
this is just one `polars` query away!

In [43]:
pois_checkins = final_sorted.explode(["pois", "geohashes"]).drop("n_checkins")

pois_checkins = (
    pois_checkins.with_columns(
        [
            rs.col("geohashes").map_elements(lambda s: s[:4], rs.String).alias("g4"),
        ]
    )
    .drop("geohashes")
    .group_by(["pois", "g4"])
    .agg([rs.col("times_sorted").flatten().alias("checkin_times")])
)

In [44]:
pois_checkins  # with this we can *efficiently* build our POI-POI spatial-temporal graphs

pois,g4,checkin_times
str,str,list[str]
"""4ad8f6f2f964a5…","""dppn""","[""2012-04-07 14:54:35"", ""2012-04-07 18:13:18"", … ""2012-12-30 18:57:21""]"
"""4a9aea62f964a5…","""dqcj""","[""2012-04-04 20:17:27"", ""2012-04-07 18:10:40"", … ""2013-09-10 16:47:33""]"
"""4d92b679b053b6…","""sxk9""","[""2012-04-04 10:17:00"", ""2012-04-05 14:10:53"", … ""2013-08-13 22:26:47""]"
"""4e5a143618388c…","""qqgu""","[""2012-09-05 18:59:37"", ""2012-09-13 17:59:58"", … ""2012-07-08 20:11:16""]"
"""4c2cdff977cfe2…","""sxk9""","[""2012-04-09 22:39:08"", ""2012-04-11 09:36:16"", … ""2013-08-21 22:40:59""]"
…,…,…
"""4b55d164f964a5…","""sxft""","[""2012-05-19 18:00:48"", ""2012-11-06 18:58:43"", … ""2013-08-24 08:53:36""]"
"""4a4c0b94f964a5…","""dpsb""","[""2012-04-07 16:05:49"", ""2012-04-14 19:15:52"", … ""2013-04-24 13:56:23""]"
"""4b5a80a4f964a5…","""xn77""","[""2012-05-04 21:09:08"", ""2012-05-04 21:09:23"", … ""2013-08-21 18:41:30""]"


In [45]:
def UTC_to_weekslot(utc: str) -> int:
    """UTC_to_weekslot converts a UTC timestamp to a weekslot.

    Parameters
    ----------
    utc : str
        A string representing a UTC timestamp.

    Returns
    -------
    int
        A weekslot in the range [0, 56).
    """

    date = datetime.datetime.strptime(utc, "%Y-%m-%d %H:%M:%S")
    week = date.weekday()
    hour = date.hour

    return week * 8 + hour // 3

Next, we want to encode all of our inputs for our neural networks, this could *probably* be done 
with polars magic, but it's too delicate and we prefer classic for-looping.

In [46]:
encoder_dict = {
    "users": LabelEncoder(),
    "pois": LabelEncoder(),
    "g2": LabelEncoder(),
    "g3": LabelEncoder(),
    "g4": LabelEncoder(),
    "g5": LabelEncoder(),
    "g6": LabelEncoder(),
}

encoded_data = {
    "users": [],
    "pois": [],
    "g2": [],
    "g3": [],
    "g4": [],
    "g5": [],
    "g6": [],
}

unique_data = {
    "users": set(),
    "pois": set(),
    "g2": set(),
    "g3": set(),
    "g4": set(),
    "g5": set(),
    "g6": set(),
}

# quick and dirty encoding:
# 1. put every unique symbol in a list
# 2. fit the respective encoder
# 3. transform the lists

for i, row in enumerate(final_sorted.iter_rows()):

    user, pois, geohashes, times_sorted, n_checkins = row

    g2 = [geo[:2] for geo in geohashes]
    g3 = [geo[:3] for geo in geohashes]
    g4 = [geo[:4] for geo in geohashes]
    g5 = [geo[:5] for geo in geohashes]
    g6 = [geo[:6] for geo in geohashes]  # redundant, but I like symmetry

    unique_data["users"].add(user)
    unique_data["pois"].update(pois)
    unique_data["g2"].update(g2)
    unique_data["g3"].update(g3)
    unique_data["g4"].update(g4)
    unique_data["g5"].update(g5)
    unique_data["g6"].update(g6)

for property, enc, data in zip(
    encoder_dict.keys(), encoder_dict.values(), unique_data.values()
):
    enc.fit(list(data))
    encoder_dict[property] = enc

In [47]:
# this could be optimized, right now it takes a while, at least we have a nice progress bar to look at

ds_size = len(final_sorted)

for i, row in tqdm(enumerate(final_sorted.iter_rows()), total=ds_size):

    user, pois, geohashes, times_sorted, n_checkins = row

    g2 = [geo[:2] for geo in geohashes]
    g3 = [geo[:3] for geo in geohashes]
    g4 = [geo[:4] for geo in geohashes]
    g5 = [geo[:5] for geo in geohashes]
    g6 = [geo[:6] for geo in geohashes]

    encoded_data["users"].append(encoder_dict["users"].transform([user])[0])
    encoded_data["pois"].append(encoder_dict["pois"].transform(pois))
    encoded_data["g2"].append(encoder_dict["g2"].transform(g2))
    encoded_data["g3"].append(encoder_dict["g3"].transform(g3))
    encoded_data["g4"].append(encoder_dict["g4"].transform(g4))
    encoded_data["g5"].append(encoder_dict["g5"].transform(g5))
    encoded_data["g6"].append(encoder_dict["g6"].transform(g6))

    # sum 1 to all values to avoid 0s
    encoded_data["users"][-1] += 1
    encoded_data["pois"][-1] += 1
    encoded_data["g2"][-1] += 1
    encoded_data["g3"][-1] += 1
    encoded_data["g4"][-1] += 1
    encoded_data["g5"][-1] += 1
    encoded_data["g6"][-1] += 1

100%|██████████| 19862/19862 [04:23<00:00, 75.24it/s]


In [48]:
# check that we left space for the padding token
min((arr.min() for arr in encoded_data["pois"]))

1

In [49]:
pois_checkins

pois,g4,checkin_times
str,str,list[str]
"""4ad8f6f2f964a5…","""dppn""","[""2012-04-07 14:54:35"", ""2012-04-07 18:13:18"", … ""2012-12-30 18:57:21""]"
"""4a9aea62f964a5…","""dqcj""","[""2012-04-04 20:17:27"", ""2012-04-07 18:10:40"", … ""2013-09-10 16:47:33""]"
"""4d92b679b053b6…","""sxk9""","[""2012-04-04 10:17:00"", ""2012-04-05 14:10:53"", … ""2013-08-13 22:26:47""]"
"""4e5a143618388c…","""qqgu""","[""2012-09-05 18:59:37"", ""2012-09-13 17:59:58"", … ""2012-07-08 20:11:16""]"
"""4c2cdff977cfe2…","""sxk9""","[""2012-04-09 22:39:08"", ""2012-04-11 09:36:16"", … ""2013-08-21 22:40:59""]"
…,…,…
"""4b55d164f964a5…","""sxft""","[""2012-05-19 18:00:48"", ""2012-11-06 18:58:43"", … ""2013-08-24 08:53:36""]"
"""4a4c0b94f964a5…","""dpsb""","[""2012-04-07 16:05:49"", ""2012-04-14 19:15:52"", … ""2013-04-24 13:56:23""]"
"""4b5a80a4f964a5…","""xn77""","[""2012-05-04 21:09:08"", ""2012-05-04 21:09:23"", … ""2013-08-21 18:41:30""]"


In [50]:
# we also encode the graph dataframe so we can build the graphs

pois_checkins = (
    pois_checkins.lazy()
    .with_columns(
        [
            rs.col("pois").map_elements(
                lambda s: encoder_dict["pois"].transform([s])[0] + 1, rs.Int64
            ),
            rs.col("g4").map_elements(
                lambda s: encoder_dict["g4"].transform([s])[0] + 1, rs.Int64
            ),  # apply utc_to_weekslot to each timestamp in the list
            rs.col("checkin_times").map_elements(
                lambda s: [UTC_to_weekslot(date) for date in s], rs.List(rs.Int64)
            ),
        ]
    )
    .sort("pois")
    .collect()
)

In [51]:
# add fictitious POI 0 to the graph, with nonexistent geohash and no timeslot, so we get a 0 row and column for the padding token
fake_datapoint = rs.DataFrame(
    {
        "pois": [0],
        "g4": [pois_checkins["g4"].max() + 42],
        "checkin_times": [[43]],
    }
)
# this is a lot of work since polars dataframes are immutable by default, we have to run a query to change the 43 into an empty list
# we NEED the 43 otherwise polars won't infer the datatype of the list

fake_datapoint = fake_datapoint.with_columns(
    [rs.col("checkin_times").map_elements(lambda s: [], rs.List(rs.Int64))]
)

pois_checkins = fake_datapoint.vstack(pois_checkins)

SchemaError: expected output type 'List(Int64)', got 'List(Null)'; set `return_dtype` to the proper datatype

In [ ]:
spatial_row = np.array(pois_checkins["g4"].to_list()).reshape(-1, 1)

In [ ]:
# outer product using equality
spatial_graph = (spatial_row == spatial_row.T).astype(np.int32)
spatial_graph[0, 0] = (
    0  # the fake g4 is still equal to itself, we suppress this equality
)
spatial_graph = torch.tensor(spatial_graph)

In [ ]:
temporal_row = pois_checkins["checkin_times"].to_list()

In [ ]:
temporal_graph = np.zeros((spatial_row.shape[0], spatial_row.shape[0]))

In [ ]:
temporal_sets = [np.array(list(set(row))) for row in temporal_row]

In [ ]:
time_sets = torch.zeros((len(temporal_sets), 56), dtype=torch.int8)

for i, r in enumerate(temporal_row):
    indices = torch.tensor(r, dtype=torch.long)
    time_sets[i, indices] = 1

In [ ]:
time_sets.shape

In [ ]:
# AND outer product

intersection = time_sets @ time_sets.T
union = time_sets.unsqueeze(1) | time_sets.unsqueeze(0)
union = union.sum(dim=2)
iou = intersection / union

In [ ]:
temporal_graph = iou >= 0.9
# cast to int
temporal_graph = temporal_graph.int()

In [ ]:
temporal_graph[0, :].sum()

We print information about the sparsity of the graphs, we note that 
the sparsity of the graphs is similar to that of the paper.

In [ ]:
temporal_density = (
    temporal_graph.sum() / (temporal_graph.shape[0] * temporal_graph.shape[1])
).item()
spatial_density = (
    spatial_graph.sum() / (spatial_graph.shape[0] * spatial_graph.shape[1])
).item()

print(f"Temporal sparsity: {(1 - temporal_density) * 100:.2f}%")

print(f"Spatial sparsity: {(1 - spatial_density) * 100:.2f}%")

## Metrics


The paper utilizes metrics that check if the target is in the top-k recommendations, we implement them here.

In [ ]:
class AccuracyAtK(nn.Module):
    def __init__(self, k: int):
        """__init__ initializes the AccuracyAtK module.

        Accuracy@k is the proportion of correct predictions in the top-k elements.

        Parameters
        ----------
        k : int
            The number of top-k elements to consider.

        """
        super().__init__()
        self.k = k

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """forward computes the accuracy at k between logits and targets.

        Parameters
        ----------
        logits : torch.Tensor
            Class probability, either (B, C) or (B, T, C)
        targets : torch.Tensor
            Ground truth class indices, either (B,) or (B, T)

        Returns
        -------
        torch.Tensor
            The accuracy at k, a scalar-tensor.
        """

        # Gotta have at least one nasty python one-liner, in memory of the old
        # programming lab 1 bachelor course
        return (
            (logits.topk(self.k, dim=-1)[1] == targets.unsqueeze(-1))
            .any(dim=-1)
            .float()
            .mean()
        )


class MeanReciprocalRank(nn.Module):

    def __init__(self):
        """__init__ initializes the MeanReciprocalRank module.

        Mean reciprocal rank is the average of the reciprocal ranks of the top-k elements.

        """
        super().__init__()

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """forward computes the mean reciprocal rank between logits and targets.

        Parameters
        ----------
        logits : torch.Tensor
            Class probability
        targets : torch.Tensor
            Ground truth class indices

        Returns
        -------
        torch.Tensor
            The mean reciprocal rank, a scalar-tensor.
        """

        _, indices = logits.topk(logits.shape[-1], dim=-1)
        ranks = (indices == targets.unsqueeze(-1)).nonzero()[:, -1].float() + 1
        return (1.0 / ranks).mean()

## Dataset and Datamodule

We then define a pytorch dataset and a custom collation function that allows us to dynamically
pad sequences to the longest one in the batch (as opposed to the longest one in the dataset)
as they are loaded during training, this gives us an edge in performance by dramatically reducing the 
sparsity of our inputs.

In [ ]:
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


def rnn_collation_fn(batch):

    users = []
    pois = []
    g2 = []
    g3 = []
    g4 = []
    g5 = []
    g6 = []

    for user, poi, geo2, geo3, geo4, geo5, geo6 in batch:
        users.append(user)
        pois.append(poi)
        g2.append(geo2)
        g3.append(geo3)
        g4.append(geo4)
        g5.append(geo5)
        g6.append(geo6)
    seq = (
        torch.tensor(users, dtype=torch.long),
        pad_sequence(pois, batch_first=True, padding_value=0),
        pad_sequence(g2, batch_first=True, padding_value=0),
        pad_sequence(g3, batch_first=True, padding_value=0),
        pad_sequence(g4, batch_first=True, padding_value=0),
        pad_sequence(g5, batch_first=True, padding_value=0),
        pad_sequence(g6, batch_first=True, padding_value=0),
    )  # build a sequence

    x = (
        seq[0],
        seq[1][:, :-1],
        seq[2][:, :-1],
        seq[3][:, :-1],
        seq[4][:, :-1],
        seq[5][:, :-1],
        seq[6][:, :-1],
    )  # omit the last one for sample

    y = (
        seq[0],
        seq[1][:, 1:],
        seq[2][:, 1:],
        seq[3][:, 1:],
        seq[4][:, 1:],
        seq[5][:, 1:],
        seq[6][:, 1:],
    )  # omit the first one for ground truth

    return x, y


class CheckinDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data["users"])

    def __getitem__(self, idx):

        x = (
            torch.tensor(encoded_data["users"][idx], dtype=torch.long),
            torch.tensor(encoded_data["pois"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g2"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g3"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g4"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g5"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g6"][idx], dtype=torch.long),
        )

        return x

In [ ]:
class CheckinModule(pl.LightningDataModule):
    def __init__(self, encoded_data, batch_size=32, workers=4):
        super().__init__()
        self.encoded_data = encoded_data
        self.batch_size = batch_size
        self.workers = workers

    def setup(self, stage=None):
        self.whole_dataset = CheckinDataset(self.encoded_data)

        l = len(self.whole_dataset)

        train_size = int(0.8 * l)
        val_size = int(0.1 * l)
        test_size = l - train_size - val_size

        # generate train, val, test datasets by random split
        self.train_dataset, self.val_dataset, self.test_dataset = (
            torch.utils.data.random_split(
                self.whole_dataset, [train_size, val_size, test_size]
            )
        )

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.workers,
            collate_fn=rnn_collation_fn,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.workers,
            collate_fn=rnn_collation_fn,
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.workers,
            collate_fn=rnn_collation_fn,
        )

    def save(self, whole_path, train_path, val_path, test_path):
        torch.save(self.whole_dataset, whole_path)
        torch.save(self.train_dataset, train_path)
        torch.save(self.val_dataset, val_path)
        torch.save(self.test_dataset, test_path)

    @staticmethod  # load without instantiating
    def load(whole_path, train_path, val_path, test_path):
        whole_dataset = torch.load(whole_path)
        train_dataset = torch.load(train_path)
        val_dataset = torch.load(val_path)
        test_dataset = torch.load(test_path)
        return whole_dataset, train_dataset, val_dataset, test_dataset

## Baseline model: LSTM

In [ ]:
@dataclass
class BaselineDimensions:
    nuser: int
    npoi: int
    g2len: int
    g3len: int
    g4len: int
    g5len: int
    g6len: int


# HMT_RN (Hierarchical Multi-Task Recurrent Network)
class HMT_RN(pl.LightningModule):
    def __init__(
        self,
        dimensions: BaselineDimensions,
        embedding_dim,
        lstm_hidden_dim,
        dropout_rate=0.9,  # 0.9 is a lot, but the paper says so.
    ):
        super(HMT_RN, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = lstm_hidden_dim
        self.dims = dimensions

        # Embedding layers one for user, one for poi and one for each G@P
        self.user_embedding = nn.Embedding(dims.nuser, hidden_dim, padding_idx=0)
        self.poi_embedding = nn.Embedding(dimensions.npoi, embedding_dim, padding_idx=0)
        self.g2_embed = nn.Embedding(dimensions.g2len, embedding_dim, padding_idx=0)
        self.g3_embed = nn.Embedding(dimensions.g3len, embedding_dim, padding_idx=0)
        self.g4_embed = nn.Embedding(dimensions.g4len, embedding_dim, padding_idx=0)
        self.g5_embed = nn.Embedding(dimensions.g5len, embedding_dim, padding_idx=0)
        self.g6_embed = nn.Embedding(dimensions.g6len, embedding_dim, padding_idx=0)

        # Dropout layer for embeddings
        self.e_drop = nn.Dropout(p=dropout_rate)

        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=embedding_dim, hidden_size=lstm_hidden_dim, batch_first=True
        )

        # Linear layers for prediction tasks
        self.linear_poi = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.npoi)
        self.linear_g2 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g2len)
        self.linear_g3 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g3len)
        self.linear_g4 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g4len)
        self.linear_g5 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g5len)
        self.linear_g6 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g6len)

        self.criterion = nn.CrossEntropyLoss()

        self.top1 = AccuracyAtK(1)
        self.top5 = AccuracyAtK(5)
        self.top10 = AccuracyAtK(10)
        self.top20 = AccuracyAtK(20)
        self.mrr = MeanReciprocalRank()

        self.apply(self.init_weights)

    def init_weights(self, w):

        if type(w) == nn.Linear:
            nn.init.kaiming_normal_(w.weight)
            nn.init.constant_(w.bias, 0)
        elif type(w) == nn.LSTM:
            for name, param in w.named_parameters():
                if "bias" in name:
                    nn.init.constant_(param, 0)
                elif "weight" in name:
                    nn.init.kaiming_normal_(param)
        elif type(w) == nn.Embedding:
            nn.init.kaiming_normal_(w.weight)
            nn.init.constant_(w.weight[0], 0)

    def forward(self, batch):
        """forward passes the batch through the model.

        Parameters
        ----------
        batch : `tuple[torch.Tensor]`
            A tuple of tensors ordered as follows:
            (users, poi, x_geoHash2, x_geoHash3, x_geoHash4, x_geoHash5, x_geoHash6)
        """

        users, poi, x_geoHash2, x_geoHash3, x_geoHash4, x_geoHash5, x_geoHash6 = batch

        B, T = poi.shape

        # make it so  that users are tiled T times
        users = users.repeat(T, 1).T
        
        e_user = self.e_drop(self.user_embedding(users))
        e_poi = self.e_drop(self.poi_embedding(poi))
        e_gap2 = self.e_drop(self.g2_embed(x_geoHash2))
        e_gap3 = self.e_drop(self.g3_embed(x_geoHash3))
        e_gap4 = self.e_drop(self.g4_embed(x_geoHash4))
        e_gap5 = self.e_drop(self.g5_embed(x_geoHash5))
        e_gap6 = self.e_drop(self.g6_embed(x_geoHash6))

        h_t, c_t = self.lstm(e_poi)

        # dense layers
        next_poi = self.linear_poi(torch.cat((h_t, e_user), dim=2))
        next_g2 = self.linear_g2(torch.cat((h_t, e_gap2), dim=2))
        next_g3 = self.linear_g3(torch.cat((h_t, e_gap3), dim=2))
        next_g4 = self.linear_g4(torch.cat((h_t, e_gap4), dim=2))
        next_g5 = self.linear_g5(torch.cat((h_t, e_gap5), dim=2))
        next_g6 = self.linear_g6(torch.cat((h_t, e_gap6), dim=2))

        return next_poi, next_g2, next_g3, next_g4, next_g5, next_g6

    def training_step(self, batch, batch_idx):
        x, y = batch
        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6
        self.log("train/loss", loss)
        self.log("train/loss_gap2", loss_gap2)
        self.log("train/loss_gap3", loss_gap3)
        self.log("train/loss_gap4", loss_gap4)
        self.log("train/loss_gap5", loss_gap5)
        self.log("train/loss_gap6", loss_gap6)
        self.log("train/loss_poi", loss_poi)

        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6

        top1_acc = self.top1(poi_pred, y[1])
        top5_acc = self.top5(poi_pred, y[1])
        top10_acc = self.top10(poi_pred, y[1])
        top20_acc = self.top20(poi_pred, y[1])
        mrr = self.mrr(poi_pred, y[1])

        self.log("val/loss", loss)
        self.log("val/loss_gap2", loss_gap2)
        self.log("val/loss_gap3", loss_gap3)
        self.log("val/loss_gap4", loss_gap4)
        self.log("val/loss_gap5", loss_gap5)
        self.log("val/loss_gap6", loss_gap6)
        self.log("val/loss_poi", loss_poi)

        # log "leaderboard" metrics
        self.log("val/top1", top1_acc)
        self.log("val/top5", top5_acc)
        self.log("val/top10", top10_acc)
        self.log("val/top20", top20_acc)
        self.log("val/mrr", mrr)

        return loss

    def configure_optimizers(self):
        # Define optimizer and scheduler
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4, amsgrad=True)
        return optimizer

## Graph Neural Network

In [ ]:
# GNN Components


class attn_LSTM(pl.LightningModule):

    def __init__(self, embedding_dim, hidden_dim):
        super(attn_LSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.W = nn.Linear(embedding_dim, 4 * hidden_dim)
        self.U = nn.Linear(hidden_dim, 4 * hidden_dim)

        self.s_W = nn.Linear(embedding_dim, 4 * hidden_dim)
        self.t_W = nn.Linear(embedding_dim, 4 * hidden_dim)

    def forward(self, x, hidden, spatial, temporal, numTimeSteps):
        h_t, c_t = hidden

        previous_h_t = h_t
        previous_c_t = c_t

        allGates_preact = (
            self.W(x) + self.U(previous_h_t) + self.s_W(spatial) + self.t_W(temporal)
        )

        input_g = allGates_preact[:, :, : self.hidden_dim].sigmoid()
        forget_g = allGates_preact[
            :, :, self.hidden_dim : 2 * self.hidden_dim
        ].sigmoid()
        output_g = allGates_preact[
            :, :, 2 * self.hidden_dim : 3 * self.hidden_dim
        ].sigmoid()
        c_t_g = allGates_preact[:, :, 3 * self.hidden_dim :].tanh()

        c_t = forget_g * previous_c_t + input_g * c_t_g
        h_t = output_g * c_t.tanh()

        batchSize = x.shape[0]
        h_t = h_t.view(batchSize, numTimeSteps, self.hidden_dim)
        c_t = c_t.view(batchSize, numTimeSteps, self.hidden_dim)

        return h_t, c_t


def get_neighbours(adj_matrix, poi):
    neigh_indices_list = []
    max_length = 0

    for batch_poi in poi:
        batch_indices = []
        for single_poi in batch_poi:
            poi_row = adj_matrix[single_poi]
            neigh_indices = torch.where(poi_row == 1)[0]
            batch_indices.append(neigh_indices)
            max_length = max(max_length, len(neigh_indices))

        neigh_indices_list.append(batch_indices)

    padded_neigh_indices_list = []
    for batch_indices in neigh_indices_list:
        padded_batch_indices = pad_sequence(
            batch_indices, batch_first=True, padding_value=0
        )
        padded_neigh_indices_list.append(padded_batch_indices)

    padded_tensor = torch.stack(padded_neigh_indices_list)

    return padded_tensor


class GRNSelfAttention(torch.nn.Module):

    def __init__(self, hidden_dim, n_heads):

        super(GRNSelfAttention, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_heads = n_heads

        self.Wp = nn.Linear(hidden_dim, hidden_dim)  # embeddings to pre-concat
        self.Wa = nn.Linear(2 * hidden_dim, hidden_dim)  # concatenation to pre-softmax

        # total size = 3 * (hidden_dim) ** 2, quadratic in embedding size

    def forward(self, poi, neighbors):
        """forward

        Parameters
        ----------
        poi: torch.Tensor
            A batched tensor of embedded POI vectors, (B x H) where H is the
            embedding dimension
        neighbors: torch.Tensor
            A batched tensor of sequences of embedded POI vectors that are extracted
            from an adjacency matrix (temporal or spatial neighbors of POI),
            (B x N x H), where N is the number of neighbours of POI, B is the
            batch size, H is the embedding dimension, and must be the same as POI

        Returns
        -------
        tuple[torch.Tensor, torch.Tensor]
          A tuple containing the self-attention weighted hadamard product of neighbour activations
          in the first index, the attention weights in the second index.
        """
        # assert len(poi.shape) == 2, f"POI tensor must be 2D, got {poi.shape} instead"
        assert (
            len(neighbors.shape) == 3
        ), f"Neighbour tensor must be 3D, got {neighbors.shape} instead"

        B, N, H = neighbors.shape

        h_poi = self.Wp(poi)
        h_n = self.Wp(neighbors)
        h_cat = torch.cat([h_poi.expand(B, N, -1), h_n], dim=2)
        h_att = F.leaky_relu(self.Wa(h_cat))

        alpha = torch.nn.functional.softmax(h_att, dim=1)

        p = torch.sum(alpha * h_n, dim=1)
        return p, alpha

In [ ]:
# GRN (Graph Recurrent Network)
class GRN(pl.LightningModule):

    def __init__(
        self,
        dims: BaselineDimensions,
        spatial_graph,
        temporal_graph,
        hidden_dim,
        n_heads,
        dropout_rate=0.9,
        device="cpu",
    ):
        super(GRN, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_heads = n_heads
        self.dims = dims

        self.spatial_graph = spatial_graph.to(device)
        self.temporal_graph = temporal_graph.to(device)

        self.spatial_attn = GRNSelfAttention(hidden_dim, n_heads)
        self.temporal_attn = GRNSelfAttention(hidden_dim, n_heads)

        self.lstm = attn_LSTM(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_rate)

        self.user_embedding = nn.Embedding(dims.nuser, hidden_dim, padding_idx=0)
        self.poi_embedding = nn.Embedding(dims.npoi, hidden_dim, padding_idx=0)
        self.g2_embed = nn.Embedding(dims.g2len, hidden_dim, padding_idx=0)
        self.g3_embed = nn.Embedding(dims.g3len, hidden_dim, padding_idx=0)
        self.g4_embed = nn.Embedding(dims.g4len, hidden_dim, padding_idx=0)
        self.g5_embed = nn.Embedding(dims.g5len, hidden_dim, padding_idx=0)
        self.g6_embed = nn.Embedding(dims.g6len, hidden_dim, padding_idx=0)

        self.linear_poi = nn.Linear(2 * hidden_dim, dims.npoi)
        self.linear_g2 = nn.Linear(2 * hidden_dim, dims.g2len)
        self.linear_g3 = nn.Linear(2 * hidden_dim, dims.g3len)
        self.linear_g4 = nn.Linear(2 * hidden_dim, dims.g4len)
        self.linear_g5 = nn.Linear(2 * hidden_dim, dims.g5len)
        self.linear_g6 = nn.Linear(2 * hidden_dim, dims.g6len)

        # extract indices from one-hot neighbor list
        self.iota = torch.arange(self.dims.npoi, requires_grad=False, device=device)

        self.criterion = nn.CrossEntropyLoss()

        self.apply(self.init_weights)

    def init_weights(self, w):
        if type(w) == nn.Linear:
            nn.init.kaiming_normal_(w.weight)
            nn.init.constant_(w.bias, 0)
        elif type(w) == nn.LSTM:
            for name, param in w.named_parameters():
                if "bias" in name:
                    nn.init.constant_(param, 0)
                elif "weight" in name:
                    nn.init.kaiming_normal_(param)
        elif type(w) == nn.Embedding:
            nn.init.kaiming_normal_(w.weight)
            nn.init.constant_(w.weight[0], 0)

    def forward(self, x):

        users, poi, x_geoHash2, x_geoHash3, x_geoHash4, x_geoHash5, x_geoHash6 = x

        B, T = poi.shape

        users = users.repeat(T, 1).T

        neighbors_spatial = self.spatial_graph[poi]
        neighbors_temporal = self.temporal_graph[poi]

        e_user = self.dropout(self.user_embedding(users))
        e_poi = self.dropout(self.poi_embedding(poi))
        e_gap2 = self.dropout(self.g2_embed(x_geoHash2))
        e_gap3 = self.dropout(self.g3_embed(x_geoHash3))
        e_gap4 = self.dropout(self.g4_embed(x_geoHash4))
        e_gap5 = self.dropout(self.g5_embed(x_geoHash5))
        e_gap6 = self.dropout(self.g6_embed(x_geoHash6))

        spatial_atts = torch.empty((B, T, self.hidden_dim), device=self.device)
        temporal_atts = torch.empty((B, T, self.hidden_dim), device=self.device)

        for b in range(B):
            for t in range(T):

                print(b, t)

                spatial_neigh = neighbors_spatial[b, t] * self.iota
                temporal_neigh = neighbors_temporal[b, t] * self.iota

                spatial_neigh = spatial_neigh[spatial_neigh != 0]
                temporal_neigh = temporal_neigh[temporal_neigh != 0]

                spatial_neigh = spatial_neigh.unsqueeze(0)
                temporal_neigh = temporal_neigh.unsqueeze(0)

                e_spatial = self.dropout(self.poi_embedding(spatial_neigh))
                e_temporal = self.dropout(self.poi_embedding(temporal_neigh))

                curr_poi = e_poi[b, t].unsqueeze(0)

                spatial_p, _ = self.spatial_attn(curr_poi, e_spatial)
                temporal_p, _ = self.temporal_attn(curr_poi, e_temporal)

                # we are not using the batch dimension, so we squeeze it
                spatial_atts[b, t] = spatial_p.squeeze()
                temporal_atts[b, t] = temporal_p.squeeze()

        # zero-init LSTM states
        h_t = torch.zeros(B, T, self.hidden_dim, device=self.device)
        c_t = torch.zeros(B, T, self.hidden_dim, device=self.device)

        h_t, c_t = self.lstm(e_poi, (h_t, c_t), spatial_atts, temporal_atts, T)
        
        #Note:the prediction of the poi depends on the embedding o the user
        next_poi = self.linear_poi(torch.cat((h_t, e_user), dim=2))
        next_g2 = self.linear_g2(torch.cat((h_t, e_gap2), dim=2))
        next_g3 = self.linear_g3(torch.cat((h_t, e_gap3), dim=2))
        next_g4 = self.linear_g4(torch.cat((h_t, e_gap4), dim=2))
        next_g5 = self.linear_g5(torch.cat((h_t, e_gap5), dim=2))
        next_g6 = self.linear_g6(torch.cat((h_t, e_gap6), dim=2))

        return next_poi, next_g2, next_g3, next_g4, next_g5, next_g6

    def training_step(self, batch, batch_idx):
        x, y = batch
        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6
        self.log("train/loss", loss)
        self.log("train/loss_gap2", loss_gap2)
        self.log("train/loss_gap3", loss_gap3)
        self.log("train/loss_gap4", loss_gap4)
        self.log("train/loss_gap5", loss_gap5)
        self.log("train/loss_gap6", loss_gap6)
        self.log("train/loss_poi", loss_poi)

        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6

        self.log("val/loss", loss)
        self.log("val/loss_gap2", loss_gap2)
        self.log("val/loss_gap3", loss_gap3)
        self.log("val/loss_gap4", loss_gap4)
        self.log("val/loss_gap5", loss_gap5)
        self.log("val/loss_gap6", loss_gap6)
        self.log("val/loss_poi", loss_poi)

        return loss

    def configure_optimizers(self):
        # Define optimizer and scheduler
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4, amsgrad=True)
        return optimizer

## Training Loops

In [ ]:
n_users = encoder_dict["users"].classes_.shape[0]
n_pois = encoder_dict["pois"].classes_.shape[0]
n_g2 = encoder_dict["g2"].classes_.shape[0]
n_g3 = encoder_dict["g3"].classes_.shape[0]
n_g4 = encoder_dict["g4"].classes_.shape[0]
n_g5 = encoder_dict["g5"].classes_.shape[0]
n_g6 = encoder_dict["g6"].classes_.shape[0]


# account for the padding token
dims = BaselineDimensions(
    n_users + 1, n_pois + 1, n_g2 + 1, n_g3 + 1, n_g4 + 1, n_g5 + 1, n_g6 + 1
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch import Trainer

TRAIN_BASELINE = True

wandb.finish()
torch.cuda.empty_cache()
# cargo-cult like stuff that is supposed to make you faster
torch.set_float32_matmul_precision("medium")
torch.backends.cudnn.benchmark = True

ds = CheckinModule(encoded_data, batch_size=32, workers=4)

wandb.init(project="trovailpoi")

classifier_baseline = HMT_RN(dims, embedding_dim=1024, lstm_hidden_dim=1024)
wandb_logger = WandbLogger(project="trovailpoi")
trainer = Trainer(
    max_epochs=40,
    accelerator="auto",
    devices=[0],
    log_every_n_steps=10,
    logger=wandb_logger,
    strategy="auto",
    callbacks=[
        torchpl.callbacks.LearningRateMonitor(logging_interval="step"),
        torchpl.callbacks.ModelCheckpoint(
            monitor="val/loss",
            mode="min",
            save_top_k=1,
            save_last=True,
            filename="best_model",
        ),
        torchpl.callbacks.EarlyStopping(
            monitor="val/loss", patience=3, min_delta=0.0005, mode="min"
        ),
    ],
)

if TRAIN_BASELINE:
    trainer.fit(model=classifier_baseline, datamodule=ds)
wandb.finish()

In [ ]:
TRAIN_GNN = False

batch_size = 60
wandb.finish()
torch.cuda.empty_cache()
# cargo-cult like stuff that is supposed to make you faster
torch.set_float32_matmul_precision("medium")
torch.backends.cudnn.benchmark = True

wandb.init(project="trovailpoi")

classifier_gnn = GRN(
    dims,
    spatial_graph,
    temporal_graph,
    hidden_dim=1024,
    n_heads=1,
    dropout_rate=0.9,
    device=device,
)
wandb_logger = WandbLogger(project="trovailpoi")
trainer = Trainer(
    max_epochs=40,
    accelerator="auto",
    devices=[0],
    log_every_n_steps=10,
    logger=wandb_logger,
    strategy="auto",
    callbacks=[
        torchpl.callbacks.LearningRateMonitor(logging_interval="step"),
        torchpl.callbacks.ModelCheckpoint(
            monitor="val/loss",
            mode="min",
            save_top_k=1,
            save_last=True,
            filename="best_model",
        ),
        torchpl.callbacks.EarlyStopping(
            monitor="val/loss", patience=3, min_delta=0.0005, mode="min"
        ),
    ],
)

if TRAIN_GNN:
    trainer.fit(model=classifier_gnn, datamodule=ds)
wandb.finish()

## Scrapbook for Experimentation

Ignore all code below, it's just for quick prototyping